In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path='/content/drive/My Drive/Colab Notebooks/Data_Mining/Toxic Comment Analysis/'

In [3]:
"""Un-Comment For Running locally out of colab
path='./dataset'
"""#Un-Comment For Running locally out of colab 

"Un-Comment For Running locally out of colab\npath='./dataset'\n"

In [4]:
import pandas as pd

In [5]:
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

In [6]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [7]:
train = train[:100000]
train.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.096120,0.00999,0.053270,0.003140,0.049670,0.008540
std,0.294757,0.09945,0.224573,0.055948,0.217263,0.092017
min,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000


In [8]:
classes = list(train.columns)[2:]
classes

['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [9]:
train[classes].sum()

toxic            9612
severe_toxic      999
obscene          5327
threat            314
insult           4967
identity_hate     854
dtype: int64

In [10]:
links = '(http://.*?\s)|(http://.*)'
ip_addr = '\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'
users = '\[\[User.*'
newline = '\\n'
print(train['comment_text'].str.contains(links).sum())
print(train['comment_text'].str.contains(ip_addr).sum())
print(train['comment_text'].str.contains(users).sum())
print(train['comment_text'].str.contains(newline).sum())

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


2766
6313
196
59357


In [11]:
def clean(comment):
  import re
  comment=comment.lower()
  comment=re.sub(links,'',comment)
  comment=re.sub(ip_addr,'',comment)
  comment=re.sub(users,'',comment)
  comment=re.sub(newline,'',comment)
  return comment

In [12]:
train['comment_text']=train['comment_text'].map(lambda i:clean(i))
test['comment_text']=test['comment_text'].map(lambda i:clean(i))

In [13]:
print(train['comment_text'].str.contains(links).sum())
print(train['comment_text'].str.contains(ip_addr).sum())
print(train['comment_text'].str.contains(users).sum())
print(train['comment_text'].str.contains(newline).sum())

/usr/local/lib/python3.6/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


0
0
0
0


In [14]:
x=train['comment_text']
y=train.iloc[:,2:8]
print(x.shape)
print(y.shape)

(100000,)
(100000, 6)


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(x,y,test_size=0.2,random_state=1)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [17]:
tfidf=TfidfVectorizer(max_features=5000,strip_accents='unicode',stop_words='english',token_pattern=r'\w{2,}')

In [18]:
tfidf.fit(X_train)
X_train_feat=tfidf.fit_transform(X_train)
X_train_feat.shape

(80000, 5000)

In [19]:
X_val_feat=tfidf.transform(X_val)
X_val_feat.shape

(20000, 5000)

In [20]:
import numpy as np

labels = pd.read_csv(path + "test_labels.csv")
labels=labels.iloc[:,1:]
sum_labels=np.sum(labels,axis=1)
idx=sum_labels>=0

In [21]:
labels_consider=labels[idx]
test=test.iloc[:,1:]
tests_consider=test[idx].values[:,0]

print(labels_consider.shape, tests_consider.shape)

(63978, 6) (63978,)


In [22]:
X_test_feat=tfidf.transform(tests_consider)
X_test_feat.shape

(63978, 5000)

In [23]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

model=LogisticRegression(C=20.0, max_iter=1000000)

In [24]:
## Binary Relevance

In [25]:
scores_roc_aoc = []

for label_name in classes:
    print('Class:', label_name)

    model.fit(X_train_feat, y_train[label_name])
    preds_train = model.predict(X_train_feat)
    train_roc_auc = roc_auc_score(y_train[label_name], preds_train)
    print('Train ROC AUC Score:', train_roc_auc)

    preds_val = model.predict(X_val_feat)
    val_roc_auc = roc_auc_score(y_val[label_name], preds_val)
    print('Val ROC AUC Score:', val_roc_auc)

    preds_test = model.predict(X_test_feat)
    test_roc_auc = roc_auc_score(labels_consider[label_name], preds_test)
    print('Test ROC AUC Score:', test_roc_auc)
    scores_roc_aoc.append(test_roc_auc)

    print()


print(np.mean(scores_roc_aoc))

Class: toxic
Train ROC AUC Score: 0.8641180051831711
Val ROC AUC Score: 0.8207180907734924
Test ROC AUC Score: 0.8417549380219865

Class: severe_toxic
Train ROC AUC Score: 0.7354769544912093
Val ROC AUC Score: 0.6133964600570482
Test ROC AUC Score: 0.6809342308240435

Class: obscene
Train ROC AUC Score: 0.8928312440198478
Val ROC AUC Score: 0.848488716614316
Test ROC AUC Score: 0.8235366774921389

Class: threat
Train ROC AUC Score: 0.8161114861718565
Val ROC AUC Score: 0.6307964702177707
Test ROC AUC Score: 0.6367507462186276

Class: insult
Train ROC AUC Score: 0.8373195677328823
Val ROC AUC Score: 0.7741099340770253
Test ROC AUC Score: 0.7691786795569605

Class: identity_hate
Train ROC AUC Score: 0.7554634080871397
Val ROC AUC Score: 0.6637757907724186
Test ROC AUC Score: 0.6410308739237967

0.732197691006259


In [101]:
## Classifier chains

In [26]:
from scipy.sparse import hstack, csr_matrix

def add_feat(x, feat):
  return hstack([x, csr_matrix(feat).T], 'csr')

In [27]:
scores_roc_aoc = []

for label_name in classes:
    print('Class:', label_name)

    model.fit(X_train_feat, y_train[label_name])
    preds_train = model.predict(X_train_feat)
    train_roc_auc = roc_auc_score(y_train[label_name], preds_train)
    print('Train ROC AUC Score:', train_roc_auc)

    preds_val = model.predict(X_val_feat)
    val_roc_auc = roc_auc_score(y_val[label_name], preds_val)
    print('Val ROC AUC Score:', val_roc_auc)

    preds_test = model.predict(X_test_feat)
    test_roc_auc = roc_auc_score(labels_consider[label_name], preds_test)
    print('Test ROC AUC Score:', test_roc_auc)
    scores_roc_aoc.append(test_roc_auc)

    X_train_feat = add_feat(X_train_feat, y_train[label_name])
    X_val_feat = add_feat(X_val_feat, y_val[label_name])
    X_test_feat = add_feat(X_test_feat, labels_consider[label_name])
    print(X_train_feat.shape)
    print(X_val_feat.shape)
    print(X_test_feat.shape)
    print()

print(np.mean(scores_roc_aoc))

Class: toxic
Train ROC AUC Score: 0.8641180051831711
Val ROC AUC Score: 0.8207180907734924
Test ROC AUC Score: 0.8417549380219865
(80000, 5001)
(20000, 5001)
(63978, 5001)

Class: severe_toxic
Train ROC AUC Score: 0.7721650129423699
Val ROC AUC Score: 0.6035140679772546
Test ROC AUC Score: 0.6590887040469968
(80000, 5002)
(20000, 5002)
(63978, 5002)

Class: obscene
Train ROC AUC Score: 0.9234533404235623
Val ROC AUC Score: 0.8675645938826668
Test ROC AUC Score: 0.8500987669308729
(80000, 5003)
(20000, 5003)
(63978, 5003)

Class: threat
Train ROC AUC Score: 0.8675074250380068
Val ROC AUC Score: 0.6390182680125203
Test ROC AUC Score: 0.6559903698573234
(80000, 5004)
(20000, 5004)
(63978, 5004)

Class: insult
Train ROC AUC Score: 0.9184486401928967
Val ROC AUC Score: 0.8533773949449052
Test ROC AUC Score: 0.8562945106040277
(80000, 5005)
(20000, 5005)
(63978, 5005)

Class: identity_hate
Train ROC AUC Score: 0.8589558179147085
Val ROC AUC Score: 0.7046630405520065
Test ROC AUC Score: 0.699